In [1]:
from __future__ import print_function
import os
print(os.listdir("./input"))

# Importing necessary libraries
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from keras.utils import to_categorical
from keras.layers import Dense, Input, Conv2D, Flatten, MaxPooling2D, Activation,Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras import backend as k
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation

import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output


['training-k', 'training-c', 'training-e.csv', 'training-b.csv', 'training-i', 'training-e', 'testing-c', 'training-b', 'testing-g', 'training-c.csv', 'training-d.csv', 'testing-auga', 'training-h', 'training-a.csv', 'testing-e', 'training-g', 'testing-a', 'testing-b', 'training-f', 'training-d', 'testing-f', 'training-j', 'testing-augc', 'testing-d', 'training-a']


Using TensorFlow backend.


In [2]:
FIG_WIDTH=20 # Width of figure
HEIGHT_PER_ROW=3 # Height of each row when showing a figure which consists of multiple rows
RESIZE_DIM= 64# The images will be resized to 28x28 pixels



In [29]:
data_dir=os.path.join('.','input')
paths_train_a=glob.glob(os.path.join(data_dir,'training-a','*.png'))
paths_train_c=glob.glob(os.path.join(data_dir,'training-c','*.png'))
paths_train_g=glob.glob(os.path.join(data_dir,'training-g','*.png'))
paths_train_h=glob.glob(os.path.join(data_dir,'training-h','*.png'))
paths_train_k=glob.glob(os.path.join(data_dir,'training-k','*.png'))
paths_train_l=glob.glob(os.path.join(data_dir,'training-l','*.png'))


paths_train_all=paths_train_a+paths_train_c+paths_train_g+paths_train_h+paths_train_k+paths_train_l


paths_test_all=glob.glob(os.path.join(data_dir,'testing-g','*.png'))

path_label_train_a=os.path.join(data_dir,'training-a.csv')
path_label_train_c=os.path.join(data_dir,'training-c.csv')
path_label_train_g=os.path.join(data_dir,'training-b.csv')
path_label_train_h=os.path.join(data_dir,'training-b.csv')
path_label_train_k=os.path.join(data_dir,'training-a.csv')
path_label_train_l=os.path.join(data_dir,'training-d.csv')




In [30]:
def get_key(path):
    # seperates the key of an image from the filepath
    key=path.split(sep=os.sep)[-1]
    return key

def get_data(paths_img,path_label=None,resize_dim=None):
    '''reads images from the filepaths, resizes them (if given), and returns them in a numpy array
    Args:
        paths_img: image filepaths
        path_label: pass image label filepaths while processing training data, defaults to None while processing testing data
        resize_dim: if given, the image is resized to resize_dim x resize_dim (optional)
    Returns:
        X: group of images
        y: categorical true labels
    '''
    X=[] # initialize empty list for resized images
    for i,path in enumerate(paths_img):
        img=cv2.imread(path,cv2.IMREAD_COLOR) # images loaded in color (BGR)
        #img = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # cnahging colorspace to GRAY
        if resize_dim is not None:
            img=cv2.resize(img,(resize_dim,resize_dim),interpolation=cv2.INTER_AREA) # resize image to 28x28
        #X.append(np.expand_dims(img,axis=2)) # expand image to 28x28x1 and append to the list.
        gaussian_3 = cv2.GaussianBlur(img, (9,9), 10.0) #unblur
        img = cv2.addWeighted(img, 1.5, gaussian_3, -0.5, 0, img)
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
        img = cv2.filter2D(img, -1, kernel)
        thresh = 200
        maxValue = 255
        #th, img = cv2.threshold(img, thresh, maxValue, cv2.THRESH_BINARY);
        ret,img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        X.append(img) # expand image to 28x28x1 and append to the list
        # display progress
        if i==len(paths_img)-1:
            end='\n'
        else: end='\r'
        print('processed {}/{}'.format(i+1,len(paths_img)),end=end)
        
    X=np.array(X) # tranform list to numpy array
    if  path_label is None:
        return X
    else:
        df = pd.read_csv(path_label) # read labels
        df=df.set_index('filename') 
        y_label=[df.loc[get_key(path)]['digit'] for path in  paths_img] # get the labels corresponding to the images
        y=to_categorical(y_label,10) # transfrom integer value to categorical variable
        return X, y
        

In [31]:
def imshow_group(X,y,y_pred=None,n_per_row=10,phase='processed'):
    '''helper function to visualize a group of images along with their categorical true labels (y) and prediction probabilities.
    Args:
        X: images
        y: categorical true labels
        y_pred: predicted class probabilities
        n_per_row: number of images per row to be plotted
        phase: If the images are plotted after resizing, pass 'processed' to phase argument. 
            It will plot the image and its true label. If the image is plotted after prediction 
            phase, pass predicted class probabilities to y_pred and 'prediction' to the phase argument. 
            It will plot the image, the true label, and it's top 3 predictions with highest probabilities.
    '''
    n_sample=len(X)
    img_dim=X.shape[1]
    j=np.ceil(n_sample/n_per_row)
    fig=plt.figure(figsize=(FIG_WIDTH,HEIGHT_PER_ROW*j))
    for i,img in enumerate(X):
        plt.subplot(j,n_per_row,i+1)
#         img_sq=np.squeeze(img,axis=2)
#         plt.imshow(img_sq,cmap='gray')
        plt.imshow(img)
        if phase=='processed':
            plt.title(np.argmax(y[i]))
        if phase=='prediction':
            top_n=3 # top 3 predictions with highest probabilities
            ind_sorted=np.argsort(y_pred[i])[::-1]
            h=img_dim+4
            for k in range(top_n):
                string='pred: {} ({:.0f}%)\n'.format(ind_sorted[k],y_pred[i,ind_sorted[k]]*100)
                plt.text(img_dim/2, h, string, horizontalalignment='center',verticalalignment='center')
                h+=4
            if y is not None:
                plt.text(img_dim/2, -4, 'true label: {}'.format(np.argmax(y[i])), 
                         horizontalalignment='center',verticalalignment='center')
        plt.axis('off')
    plt.show()

In [32]:
def create_submission(predictions,keys,path):
    result = pd.DataFrame(
        predictions,
        columns=['label'],
        index=keys
        )
    result.index.name='key'
    result.to_csv(path, index=True)

In [33]:
X_train_a,y_train_a=get_data(paths_train_a,path_label_train_a,resize_dim=RESIZE_DIM)
X_train_c,y_train_c=get_data(paths_train_c,path_label_train_c,resize_dim=RESIZE_DIM)
X_train_g,y_train_g=get_data(paths_train_g,path_label_train_g,resize_dim=RESIZE_DIM)
X_train_h,y_train_h=get_data(paths_train_h,path_label_train_h,resize_dim=RESIZE_DIM)
X_train_k,y_train_k=get_data(paths_train_k,path_label_train_k,resize_dim=RESIZE_DIM)
X_train_l,y_train_l=get_data(paths_train_l,path_label_train_l,resize_dim=RESIZE_DIM)


processed 19702/19702
processed 24298/24298
processed 359/359
processed 359/359
processed 19702/19702
processed 10908/10908


In [35]:
X_train_all=np.concatenate((X_train_a,X_train_c,X_train_g,X_train_h, X_train_k,X_train_l),axis=0)
y_train_all=np.concatenate((y_train_a,y_train_c,y_train_g,y_train_h,y_train_k,y_train_l),axis=0)
X_train_all.shape, y_train_all.shape

((75328, 64, 64), (75328, 10))

In [36]:
X_test_g=get_data(paths_test_all,resize_dim=RESIZE_DIM)
X_test_all=X_test_g

processed 20/20


In [37]:
X_tshow_all=X_test_all
X_tshow_all.shape

(20, 64, 64)

In [38]:
X_train_all = X_train_all.reshape(X_train_all.shape[0],64, 64,1).astype('float32')
X_test_all = X_test_all.reshape(X_test_all.shape[0],64, 64,1).astype('float32')


In [39]:

X_train_all.shape

(75328, 64, 64, 1)

In [40]:
X_train_all = X_train_all / 255
X_test_all = X_test_all / 255


In [41]:
indices=list(range(len(X_train_all)))
np.random.seed(42)
np.random.shuffle(indices)

In [42]:

ind=int(len(indices)*0.80)
# train data
X_train=X_train_all[indices[:ind]] 
y_train=y_train_all[indices[:ind]]
# validation data
X_val=X_train_all[indices[-(len(indices)-ind):]] 
y_val=y_train_all[indices[-(len(indices)-ind):]]




In [21]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    #    featurewise_center=True,
   #     samplewise_center=True,
  #      featurewise_std_normalization=True,
 #       samplewise_std_normalization=True,
#        zca_whitening=True, 
#        zca_epsilon=1e-06,
        rotation_range=45,
        width_shift_range=0.2,
        height_shift_range=0.2,
#        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
 #       horizontal_flip=True,
    #    fill_mode='nearest'
        )

In [22]:

datagen.fit(X_train)
datagen.fit(X_val)

In [46]:
from keras.optimizers import SGD, Adam
def my_model(img_size=64,channels=1):
    model = Sequential()
    input_shape = (img_size,img_size,channels)
    model.add(Conv2D(32, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
    model.add(Conv2D(32, (5, 5), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(128, (3, 3),activation='relu',padding='same'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.1))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    optmzr = SGD(lr=0.1, clipnorm=5.)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
    # UNCOMMENT THIS TO VIEW THE ARCHITECTURE
    #model.summary()

    return model

In [47]:
model = my_model()
model.summary()
path_model='model_filter.h5' # save model at this location after each epoch
# K.tensorflow_backend.clear_session() # destroys the current graph and builds a new one
model=my_model() # create the model
K.set_value(model.optimizer.lr,1e-4) # set the learning rate
# fit the model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 64, 64, 32)        832       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 64, 64, 32)        25632     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 32, 32, 128)       36992     
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 32, 32, 128)       147584    
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 16, 16, 256)       295168    
__________

In [52]:
h=model.fit(x=X_train,     
            y=y_train, 
            batch_size=256, 
            epochs=1,
            verbose=1, 
            validation_data=(X_val,y_val),
            shuffle=True,
            callbacks=[
                ModelCheckpoint(filepath=path_model),
            ]
            )


Train on 60262 samples, validate on 15066 samples
Epoch 1/1
60262/60262 [==============================] - 52s 868us/step - loss: 0.2129 - acc: 0.9273 - val_loss: 0.2032 - val_acc: 0.9348


In [53]:
predictions_prob = model.predict(X_test_all)

In [54]:
labels=[np.argmax(pred) for pred in predictions_prob]

In [55]:
keys=[get_key(path) for path in paths_test_all ]

In [56]:
create_submission(predictions=labels,keys=keys,path='box_pred_1.csv')